In [6]:
!kaggle datasets download -d dhruvildave/ookla-internet-speed-dataset -f "2021-q1/2021-01-01_performance_mobile_tiles.parquet"
!unzip 2021-01-01_performance_mobile_tiles.parquet.zip
!rm 2021-01-01_performance_mobile_tiles.parquet.zip

2021-01-01_performance_mobile_tiles.parquet.zip: Skipping, found more recently modified local copy (use --force to force download)
Archive:  2021-01-01_performance_mobile_tiles.parquet.zip
  inflating: 2021-01-01_performance_mobile_tiles.parquet  


In [3]:
# initlize pyspark
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("airline-delay-analysis").getOrCreate()

Picked up JAVA_TOOL_OPTIONS:  -Xmx3435m
Picked up JAVA_TOOL_OPTIONS:  -Xmx3435m
22/05/13 14:35:26 WARN Utils: Your hostname, fawazalesay-pysparkairl-sdsd8mdblyh resolves to a loopback address: 127.0.0.1; using 10.0.5.2 instead (on interface ceth0)
22/05/13 14:35:26 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Using Spark's default log4j profile: org/apache/spark/log4j-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
22/05/13 14:35:27 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
22/05/13 14:35:28 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.


In [4]:
# Needed to make Jupyter work with Gitpod
import plotly.io as pio
pio.renderers.default = 'iframe_connected'

In [5]:
# Read the data into a dataframe and print the schema
df = spark.read.parquet("2021-01-01_performance_mobile_tiles.parquet")
df.printSchema()

AnalysisException: Path does not exist: file:/workspace/pyspark-airline-delay-classification/2021-01-01_performance_mobile_tiles.parquet